In [ ]:
#Install Packages

!pip3 install mnist
!pip3 install keras

import numpy as np
import pandas as pd
import networkx as nx
import scipy
import numba
import pickle
import gzip

from scipy import spatial
from scipy import optimize
import matplotlib.pyplot as plt
from sklearn.manifold import SpectralEmbedding



from sklearn import datasets,metrics,preprocessing
from sklearn.datasets import fetch_20newsgroups, fetch_openml
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, ParameterGrid
#from keras.datasets import cifar10

from matplotlib.colors import ListedColormap
from matplotlib import cm


from joblib import Parallel, delayed

import mnist
from sklearn import datasets,metrics
from tqdm import tqdm

from pynndescent import NNDescent
#from umap import UMAP

from sklearn.cluster import KMeans

In [ ]:
# Load full MNIST dataset
MNIST = fetch_openml('mnist_784', version=1) 
MNIST_X_train = MNIST.data #np.vstack((mnist.train_images(), mnist.test_images()))
MNIST_n_samples = len(MNIST_X_train)
MNIST_y_train = MNIST.target.astype(int) #np.vstack((mnist.train_labels().reshape(-1, 1), mnist.test_labels().reshape(-1, 1)))


#Load 20 NewsGroup
vectorizer = CountVectorizer(min_df=5, stop_words='english') 

newsgroups_train = fetch_20newsgroups(subset='all')
newsgroups_X_train = vectorizer.fit_transform(newsgroups_train.data).toarray()
newsgroups_n_train = len(newsgroups_X_train)
newsgroups_y_train = newsgroups_train.target

#Load FMNIST
fmnist = fetch_openml('Fashion-MNIST', version=1)
#print(fmnist)
f_mnist_X = fmnist.data
f_mnist_n = len(f_mnist_X)
f_mnist_y = fmnist.target.astype(int)

#Load AG. Files downloaded at http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html
AG_train_data = pd.read_csv('data/AG/train.csv')
AG_test_data = pd.read_csv('data/AG/test.csv')
AG_data = pd.concat([AG_train_data, AG_test_data], axis=0)
AG_data['text'] = AG_data['Title'] + ' ' + AG_data['Description'] 
AG_data = AG_data.drop(columns=['Title', 'Description'])

AG_X_train =  AG_data['text'].to_numpy()
AG_y_train =  AG_data['Class Index'].to_numpy()
AG_X_train = vectorizer.fit_transform(AG_X_train)



In [ ]:
X = newsgroups_X_train
y = newsgroups_y_train
print(X.shape)

In [ ]:
# Random State
from sklearn.utils import check_random_state
random_state = check_random_state(0)

# Distance Metric to Use
metric = 'jaccard'

# number of neighbors for computing k-neighbor graph
n_neighbors = 30

# new number of neighbors to search for
new_n_neighbors = 30

### Calculate a,b hyperparams given the min_dist

In [ ]:
from umap.umap_ import find_ab_params

min_dist = 0.1
a, b = find_ab_params(1.0, min_dist)


### Calculate the weighted nearest neighbor graph 

In [ ]:
from umap.umap_ import nearest_neighbors

knn_indices, knn_dists, knn_search_index = nearest_neighbors(
    X,
    n_neighbors=n_neighbors,
    metric = metric,
    metric_kwds = {},
    angular=False,
    random_state = random_state,
    low_memory=True,
    use_pynndescent=True,
    n_jobs=1,
    verbose=True,
)



In [ ]:
#Calculate min spanning tree

def min_spanning_tree(knn_indices, knn_dists, n_neighbors, threshold):
  
  rows = np.zeros(knn_indices.shape[0] * n_neighbors, dtype=np.int32)
  cols = np.zeros(knn_indices.shape[0] * n_neighbors, dtype=np.int32)
  vals = np.zeros(knn_indices.shape[0] * n_neighbors, dtype=np.float32)
  
  pos = 0
  for i, indices in enumerate(knn_indices):
    for j, index in enumerate(indices[:threshold]):
      if index == -1:
        continue
      rows[pos] = i 
      cols[pos] = index
      vals[pos] = knn_dists[i][j]
      pos += 1
  
  matrix = scipy.sparse.csr_matrix((vals, (rows, cols)), shape=(knn_indices.shape[0], knn_indices.shape[0]))
  Tcsr = scipy.sparse.csgraph.minimum_spanning_tree(matrix)
  
  Tcsr = scipy.sparse.coo_matrix(Tcsr)
  weights_tuples = zip(Tcsr.row, Tcsr.col, Tcsr.data)
  

  sorted_weights_tuples = sorted(weights_tuples, key=lambda tup: tup[2])
  
  return sorted_weights_tuples 

In [ ]:
import copy
import heapq

def create_connected_graph(mutual_nn, total_mutual_nn, knn_indices, knn_dists, n_neighbors, connectivity):
  connected_mnn = copy.deepcopy(mutual_nn)
  
  if connectivity == "nearest":
    for i in range(len(knn_indices)): 
      if len(mutual_nn[i]) == 0:
        first_nn = knn_indices[i][1]
        if first_nn != -1:
          connected_mnn[i].add(first_nn) 
          connected_mnn[first_nn].add(i) 
          total_mutual_nn += 1
    return connected_mnn

            
      
  #Create graph for mutual NN
  rows = np.zeros(total_mutual_nn, dtype=np.int32)
  cols = np.zeros(total_mutual_nn, dtype=np.int32)
  vals = np.zeros(total_mutual_nn, dtype=np.float32)
  pos = 0
  for i in connected_mnn:
    for j in connected_mnn[i]:
      rows[pos] = i 
      cols[pos] = j
      vals[pos] = 1
      pos += 1
  graph = scipy.sparse.csr_matrix((vals, (rows, cols)), shape=(knn_indices.shape[0], knn_indices.shape[0]))
  
  
  #Find number of connected components
  n_components, labels = scipy.sparse.csgraph.connected_components(csgraph=graph, directed=True, return_labels=True, connection= 'strong')
  print(n_components)
  label_mapping = {i:[] for i in range(n_components)}

  for index, component in enumerate(labels):
    label_mapping[component].append(index)



  #Find the min spanning tree with KNN
  sorted_weights_tuples = min_spanning_tree(knn_indices, knn_dists, n_neighbors, n_neighbors)
  

  #Add edges until graph is connected
  for pos,(i,j,v) in enumerate(sorted_weights_tuples):

    if connectivity == "full_tree":
      connected_mnn[i].add(j)
      connected_mnn[j].add(i) 
      
      
    elif connectivity == "min_tree" and labels[i] != labels[j]:
      if len(label_mapping[labels[i]]) < len(label_mapping[labels[j]]):
        i, j = j, i
        
      connected_mnn[i].add(j)
      connected_mnn[j].add(i)
      j_pos = label_mapping[labels[j]]
      labels[j_pos] = labels[i]
      label_mapping[labels[i]].extend(j_pos)

  return connected_mnn  
  


In [ ]:

#Search to find adjacent neighbors 
def find_new_nn(knn_indices, knn_dists, knn_indices_pos, connected_mnn, n_neighbors_max, verbose=False):
  
  new_knn_dists= [] 
  new_knn_indices = []
  
  for i in range(len(knn_indices)): 
    #print(i)
    min_distances = []
    min_indices = []
    #Initialize vars
    heap = [(0,0,i)]
    mapping = {}
          
    seen = set()
    heapq.heapify(heap) 
    while(len(min_distances) < n_neighbors_max and len(heap) >0):
      dist, hop, nn = heapq.heappop(heap)
      if nn == -1:
        continue
      #For adjacent, only considering one hop away
      if nn not in seen and hop <= 1:
        min_distances.append(dist)
        min_indices.append(nn)
        seen.add(nn)
        neighbor = connected_mnn[nn]
        
        for nn_nn in neighbor:
          if nn_nn not in seen and hop <= 0:
            distance = 0
            if nn_nn in knn_indices_pos[nn]:
              pos = knn_indices_pos[nn][nn_nn]
              distance = knn_dists[nn][pos] 
            else:
              pos = knn_indices_pos[nn_nn][nn]
              distance = knn_dists[nn_nn][pos] 
            distance += dist
            
            if nn_nn not in mapping:
              mapping[nn_nn] = distance
              heapq.heappush(heap, (distance, hop+1, nn_nn))
            elif mapping[nn_nn] > distance:
              mapping[nn_nn] = distance
              heapq.heappush(heap, (distance, hop+1, nn_nn))
    
    if len(min_distances) < n_neighbors_max:
      for j in range(n_neighbors_max-len(min_distances)):
        min_indices.append(-1)
        min_distances.append(np.inf)
    
    new_knn_dists.append(min_distances)
    new_knn_indices.append(min_indices)
    
    if verbose and i % int(len(knn_dists) / 10) == 0:
      print("\tcompleted ", i, " / ", len(knn_dists), "epochs")
  return new_knn_dists, new_knn_indices



#Calculate the connected mutual nn graph
def mutual_nn_nearest(knn_indices, knn_dists, n_neighbors, n_neighbors_max, connectivity="min_tree", verbose=False):
  mutual_nn = {}
  nearest_n= {}

  knn_indices_pos = [None] * len(knn_indices)

  total = 0
  
  for i, top_vals in enumerate(knn_indices):
    nearest_n[i] = set(top_vals)
    knn_indices_pos[i] = {}
    for pos, nn in enumerate(top_vals):
      knn_indices_pos[i][nn] = pos
  
  total_mutual_nn = 0
  for i, top_vals in enumerate(knn_indices):
    mutual_nn[i] = set()
    for ind, nn in enumerate(top_vals):
      if nn != -1 and (i in nearest_n[nn] and i != nn):
        mutual_nn[i].add(nn)
        total_mutual_nn += 1

  
  connected_mnn = create_connected_graph(mutual_nn, total_mutual_nn, knn_indices, knn_dists, n_neighbors, connectivity )
  new_knn_dists, new_knn_indices = find_new_nn(knn_indices, knn_dists, knn_indices_pos, connected_mnn, n_neighbors_max, verbose)

  
  return connected_mnn, mutual_nn, np.array(new_knn_indices), np.array(new_knn_dists)  

    
connected_mnn,mutual_nn, new_knn_indices, new_knn_dists  = mutual_nn_nearest(knn_indices, knn_dists, n_neighbors, new_n_neighbors, connectivity= "nearest" , verbose=True)

### Calculate the Fuzzy Simplicial Set

In [ ]:
from umap.umap_ import compute_membership_strengths

INT32_MIN = np.iinfo(np.int32).min + 1
INT32_MAX = np.iinfo(np.int32).max - 1

SMOOTH_K_TOLERANCE = 1e-5
MIN_K_DIST_SCALE = 1e-3
NPY_INFINITY = np.inf

@numba.njit(
    locals={
        "psum": numba.types.float32,
        "lo": numba.types.float32,
        "mid": numba.types.float32,
        "hi": numba.types.float32,
    },
    fastmath=True,
)  

#Modified to use a variable k as explained paper
def smooth_knn_dist(distances, k, n_iter=64, local_connectivity=1.0, bandwidth=1.0):

    #target = np.log2(k) * bandwidth
    rho = np.zeros(distances.shape[0], dtype=np.float32)
    result = np.zeros(distances.shape[0], dtype=np.float32)

    mean_distances = np.mean(distances)

    for i in range(distances.shape[0]):

        
        lo = 0.0
        hi = NPY_INFINITY
        mid = 1.0

        # TODO: This is very inefficient, but will do for now. FIXME
        ith_distances = distances[i]
        non_zero_dists = ith_distances[ith_distances > 0.0]
        
        # New k
        non_inf_dists = ith_distances[ith_distances != np.inf]
        target = np.log2(len(non_inf_dists)) * bandwidth
        #print(target)
        
        
        if non_zero_dists.shape[0] >= local_connectivity:
            index = int(np.floor(local_connectivity))
            interpolation = local_connectivity - index
            if index > 0:
                rho[i] = non_zero_dists[index - 1]
                if interpolation > SMOOTH_K_TOLERANCE:
                    rho[i] += interpolation * (
                        non_zero_dists[index] - non_zero_dists[index - 1]
                    )
            else:
                rho[i] = interpolation * non_zero_dists[0]
        elif non_zero_dists.shape[0] > 0:
            rho[i] = np.max(non_zero_dists)

        for n in range(n_iter):

            psum = 0.0
            for j in range(1, distances.shape[1]):
                d = distances[i, j] - rho[i]
                if d > 0:
                    psum += np.exp(-(d / mid))
                else:
                    psum += 1.0

            if np.fabs(psum - target) < SMOOTH_K_TOLERANCE:
                break

            if psum > target:
                hi = mid
                mid = (lo + hi) / 2.0
            else:
                lo = mid
                if hi == NPY_INFINITY:
                    mid *= 2
                else:
                    mid = (lo + hi) / 2.0

        result[i] = mid

        # TODO: This is very inefficient, but will do for now. FIXME
        if rho[i] > 0.0:
            mean_ith_distances = np.mean(ith_distances)
            if result[i] < MIN_K_DIST_SCALE * mean_ith_distances:
                result[i] = MIN_K_DIST_SCALE * mean_ith_distances
        else:
            if result[i] < MIN_K_DIST_SCALE * mean_distances:
                result[i] = MIN_K_DIST_SCALE * mean_distances

    return result, rho

#Calculate fuzzy_simplicial_set with new variable k
def fuzzy_simplicial_set(
    X,
    n_neighbors,
    random_state,
    metric,
    metric_kwds={},
    knn_indices=None,
    knn_dists=None,
    angular=False,
    set_op_mix_ratio=1.0,
    local_connectivity=1.0,
    apply_set_operations=True,
    verbose=False,
    return_dists=None,
):
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
            X,
            n_neighbors,
            metric,
            metric_kwds,
            angular,
            random_state,
            verbose=verbose,
        )

    knn_dists = knn_dists.astype(np.float32)

    sigmas, rhos = smooth_knn_dist(
        knn_dists,
        float(n_neighbors),
        local_connectivity=float(local_connectivity),
    )

    rows, cols, vals, dists = compute_membership_strengths(
        knn_indices, knn_dists, sigmas, rhos, return_dists
    )

    result = scipy.sparse.coo_matrix(
        (vals, (rows, cols)), shape=(X.shape[0], X.shape[0])
    )
    result.eliminate_zeros()

    if apply_set_operations:
        transpose = result.transpose()

        prod_matrix = result.multiply(transpose)

        result = (
            set_op_mix_ratio * (result + transpose - prod_matrix)
            + (1.0 - set_op_mix_ratio) * prod_matrix
        )

    result.eliminate_zeros()

    if return_dists is None:
        return result, sigmas, rhos
    else:
        if return_dists:
            dmat = scipy.sparse.coo_matrix(
                (dists, (rows, cols)), shape=(X.shape[0], X.shape[0])
            )

            dists = dmat.maximum(dmat.transpose()).todok()
        else:
            dists = None

        return result, sigmas, rhos, dists  


In [ ]:
# build fuzzy_simplicial_set
P, sigmas, rhos = fuzzy_simplicial_set(
    X = X,
    n_neighbors = new_n_neighbors,
    metric = metric,
    random_state = random_state,
    knn_indices= new_knn_indices,
    knn_dists = new_knn_dists,
)

###  Find the low dimensional representation

In [ ]:
from umap.umap_ import simplicial_set_embedding
from umap.umap_ import dist
#Dimensionality of the low dimensional representation
n_components = 2

negative_sample_rate = 5

In [ ]:
embeddings , aux_data = simplicial_set_embedding(
    data = X,
    graph = P,
    n_components = n_components,
    initial_alpha = 1.0,
    a = a,
    b = b,
    gamma = 1.0,
    negative_sample_rate = negative_sample_rate,
    n_epochs = -1,
    init = "spectral",
    random_state = check_random_state(0),
    metric = metric,
    metric_kwds = {},
    densmap = False,
    densmap_kwds = {},
    output_dens = False,
    output_metric= dist.named_distances_with_gradients["euclidean"],
    output_metric_kwds={},
    euclidean_output=True,
    parallel=False,
    verbose=True,
)



In [ ]:
#plot for 2d
colors = []
colors += cm.get_cmap("Set3").colors
colors += cm.get_cmap("Set2").colors
my_cmap = ListedColormap(colors)
#print(my_cmap)
plt.rcParams.update(plt.rcParamsDefault)
plt.scatter(embeddings[:, 0], embeddings[:, 1], c =y.astype(int),cmap="Spectral",  s = .1)
plt.show()

In [ ]:
#Metrics calculated 


from scipy.optimize import linear_sum_assignment as linear_assignment
import seaborn as sns; sns.set()


def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)
  
def accurary_score(y_true, y_pred):
  cm = metrics.confusion_matrix(y_true, y_pred)
  indexes = linear_assignment(_make_cost_m(cm))
  cm2 = cm[indexes[0], indexes[1]]
  return np.sum(cm2) / np.sum(cm)
  
  

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix) 


# Evaluation function
def evaluate(y, y_pred):
    accuracy = [accurary_score(y, x) for x in  y_pred]
    purity = [purity_score(y, x) for x in  y_pred]
    adjusted_mutual_info_score = [metrics.adjusted_mutual_info_score(y, x) for x in  y_pred]
    adjusted_rand_score = [metrics.adjusted_rand_score(y, x) for x in  y_pred]
    normalized_mutual_info_score = [metrics.normalized_mutual_info_score(y, x) for x in  y_pred]
    homogeneity_score = [metrics.homogeneity_score(y, x) for x in  y_pred]
    completeness_score = [metrics.completeness_score(y, x) for x in  y_pred]
    v_measure_score = [metrics.v_measure_score(y, x) for x in  y_pred]
    

    print("Accuracy Score: %0.3f" % np.mean(accuracy))
    print("Accuracy STD: %0.3f" % np.std(accuracy))
    print("Purity Score: %0.3f" % np.mean(purity))
    print("Purity STD: %0.3f" % np.std(purity))
      
    print("Adjusted Mutual Information Score: %0.3f" % np.mean(adjusted_mutual_info_score))
    print("Adjusted Rand Index Score: %0.3f" % np.mean(adjusted_rand_score))
    print("Normalized Mutual Information Score: %0.3f" % np.mean(normalized_mutual_info_score))
    print("Normalized Mutual Information STD: %0.3f" % np.std(normalized_mutual_info_score))
    
    print("Homogeneity: %0.3f" % np.mean(homogeneity_score))
    print("Completeness: %0.3f" % np.mean(completeness_score))
    print("V-measure: %0.3f" % np.mean(v_measure_score))
    
    print()

In [ ]:
# k-Means clustering
def kmeans(X, n_clusters, i):
  kmeans = KMeans(n_clusters=n_clusters, random_state=i)
  y_pred = kmeans.fit_predict(X)
  return y_pred

def agglo(X, n_clusters):
  agglo = AgglomerativeClustering(n_clusters=n_clusters, linkage="ward") #KMeans(n_clusters=n_clusters, random_state=i)
  y_pred = agglo.fit_predict(X)
  return y_pred_train

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#Bootstrap function to do hyperparam search
def full_umap(X, metric, min_dist, dim, init_nn, to_add_nn, connectivity, rand_state, y):
  
  random_state = check_random_state(rand_state)
  
  a, b = find_ab_params(1.0, min_dist)
  
  knn_indices, knn_dists, knn_search_index = nearest_neighbors(
    X,
    n_neighbors=init_nn,
    metric = metric,
    metric_kwds = {},
    angular=False,
    random_state = random_state,
    low_memory=True,
    use_pynndescent=True,
    n_jobs=1,
    verbose=False,
  )
  
  
  if connectivity == "org":
    new_knn_indices, new_knn_dists = knn_indices, knn_dists
  else :
    connected_mnn, mutual_nn, new_knn_indices, new_knn_dists  = mutual_nn_nearest(knn_indices, knn_dists, init_nn, to_add_nn, connectivity=connectivity)
  
  
  
  P, sigmas, rhos = fuzzy_simplicial_set(
    X = X,
    n_neighbors = to_add_nn,
    metric = metric,
    random_state = random_state,
    knn_indices= new_knn_indices,
    knn_dists = new_knn_dists,
  )

  embeddings, aux_data = simplicial_set_embedding(
    data = X,
    graph = P,
    n_components = dim,
    initial_alpha = 1,
    a = a,
    b = b,
    gamma = 1.0,
    negative_sample_rate = 5,
    n_epochs = 200,
    init = "spectral",
    random_state = check_random_state(rand_state),
    metric = metric,
    metric_kwds = {},
    densmap = False,
    densmap_kwds = {},
    output_dens = False,
    output_metric= dist.named_distances_with_gradients["euclidean"],
    output_metric_kwds={},
    euclidean_output=True,
    parallel=False,
    verbose=False,
  )
  embeddings = np.nan_to_num(embeddings)
  return embeddings

In [ ]:

def cv_score(g, X, y, n_clusters, dim):
  avg_score = []
  for i in range(5):
    X_r  = full_umap(X, g['metric'], g['min_dist'], dim, g['n_neighbors'], g['n_neighbors'],  g['connectivity'], i, y)
    y_pred = kmeans(X_r, n_clusters, i)
    avg_score.append(metrics.normalized_mutual_info_score(y, y_pred))
  return (np.mean(avg_score), g)


  
def dimension_reduce(dimensions, X, y, n_clusters):
  param_grid = [{
        "n_neighbors": [10, 15, 20, 25, 30, 35, 50],
        "connectivity":[ "nearest", "min_tree", "full_tree"],
        "metric": ['cosine'],
        "min_dist": np.linspace(0, 1, 11)
    }]
  param_grid = ParameterGrid(param_grid)
  
  
  for dim in dimensions:
    print("Reduce")
    results = []
    results = np.array(Parallel(n_jobs=12)(delayed(cv_score)(g, X, y, n_clusters, dim)  for g in tqdm(param_grid))) 
    best_param = sorted(results, key=lambda t: t[0], reverse=True)[:1]
    best_param = best_param[0][1] 
    print(best_param)
    y_pred_train = []
    for i in range(5):
      X_umap = full_umap(X, best_param['metric'], best_param['min_dist'], dim, best_param['n_neighbors'], best_param['n_neighbors'], best_param['connectivity'], i, y)
      y_pred = kmeans(X_umap, n_clusters, i)
      y_pred_train.append(y_pred)
    print("KMeans dimension reduce: " + str(dim))
    evaluate(y,y_pred_train)

    
    print("Reduce Nearest")
    results_reduced = []
    for r in results:
      if r[1]["connectivity"] == "nearest":
        results_reduced.append(r)
        
    results_reduced = np.array(results_reduced)
    
    best_param = sorted(results_reduced, key=lambda t: t[0], reverse=True)[:1]
    best_param = best_param[0][1] 
    print(best_param)
    y_pred_train = []
    for i in range(5):
      X_umap = full_umap(X, best_param['metric'], best_param['min_dist'], dim, best_param['n_neighbors'], best_param['n_neighbors'], best_param['connectivity'], i, y)
      y_pred = kmeans(X_umap, n_clusters, i)
      y_pred_train.append(y_pred)
    print("KMeans dimension reduce: " + str(dim))
    evaluate(y,y_pred_train)
    
    
    
    print("Reduce Full Tree")
    results_reduced = []
    for r in results:
      if r[1]["connectivity"] == "full_tree":
        results_reduced.append(r)
        
    results_reduced = np.array(results_reduced)
    
    best_param = sorted(results_reduced, key=lambda t: t[0], reverse=True)[:1]
    best_param = best_param[0][1] 
    print(best_param)
    y_pred_train = []
    for i in range(5):
      X_umap = full_umap(X, best_param['metric'], best_param['min_dist'], dim, best_param['n_neighbors'], best_param['n_neighbors'], best_param['connectivity'], i, y)
      y_pred = kmeans(X_umap, n_clusters, i)
      y_pred_train.append(y_pred)
    print("KMeans dimension reduce: " + str(dim))
    evaluate(y,y_pred_train)
    
    
    
    print("Reduce Min Tree")
    results_reduced = []
    for r in results:
      if r[1]["connectivity"] == "min_tree":
        results_reduced.append(r)
        
    results_reduced = np.array(results_reduced)
    
    best_param = sorted(results_reduced, key=lambda t: t[0], reverse=True)[:1]
    best_param = best_param[0][1] 
    print(best_param)
    y_pred_train = []
    for i in range(5):
      X_umap = full_umap(X, best_param['metric'], best_param['min_dist'], dim, best_param['n_neighbors'], best_param['n_neighbors'], best_param['connectivity'], i, y)
      y_pred = kmeans(X_umap, n_clusters, i)
      y_pred_train.append(y_pred)
    print("KMeans dimension reduce: " + str(dim))
    evaluate(y,y_pred_train)
    
    

  
      
      
dimension_reduce([2, 64], X, y, 20)      